In [1]:
print('Hello World !')

Hello World !


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\ataha\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [18]:
import os
directory = os.getcwd()
print(directory)

f:\Mlops\chatbot_indian_recipes\research


In [4]:
PINECONE_API_KEY = "2fe5cb16-2c78-4e57-99e8-f373adb75ba9"
PINECONE_API_ENV = "gcp-starter"

In [16]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [19]:
extracted_data = load_pdf("f:/Mlops/chatbot_indian_recipes/data/")

In [21]:
#extracted_data

In [22]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [23]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 327


In [25]:
#text_chunks

In [27]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [28]:
embeddings = download_hugging_face_embeddings()

In [29]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [30]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [32]:
#query_result

In [33]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="langchain-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [35]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "How to make Bread Pakora ?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='3\nBread Pakora', metadata={}), Document(page_content='ingredients into wheat flour and make a \ndough.\n3. Cover dough with wet cloth and keep it \naside for 30 minutes.\n4. Divide the dough into 4 equal balls, roll it \nand place on a hot tawa and wait till slightly \ncooked.\n5. Turn and wait till brown marks appear.\n6. Apply some oil, turn and shallow-fry from \nboth sides.MethodEnergy (Kcal)   310\nProtein (g)    6.4\nFat (g)    2.02\nMinerals (g)   1.59\nFibre (g)    1.23\nCalcium (mg)   35.95\nIron (mg)    2.69\nCarotene (μg)   729.35', metadata={}), Document(page_content='2Bread Pakora\nNutritive value per serving Ingredients    Weight   Measure/No.\nBread     200 g    6 slices\nBengal gram flour   100 g    1 cup\nRefined oil (absorbed)   50 ml    5 dsp\nCurd      10 g    1 dsp\nGreen chilli paste   4 g     1 tsp\nCoriander powder   2 g    1½ tsp\nGinger paste    2 g    ½ tsp\nGarlic paste    2 g    ½ tsp\nRed chilli powder   1 g    ½ tsp\nGaram 

In [36]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [37]:
# Chain type prompt
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [46]:
# loading LLM model
llm=CTransformers(model="f:/Mlops/chatbot_indian_recipes/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [47]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [48]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

PineconeProtocolError: Failed to connect; did you specify the correct index name?